In [ ]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch
from tqdm import tqdm

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
torch.set_default_device(device)
print(torch.cuda.get_device_name(torch.cuda.current_device()))

def replace_text(row):
  print(row)
  text = row.iloc[0]
  labels = row.iloc[1]
  for entity in labels:
    if entity['entity_group'] == 'PER':
      text = text.replace(entity['word'], "<PN>")
  return text

df = pd.read_csv("data/sportsQnA.csv", dtype="string").dropna()

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")
model.to(device)

pl = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='average')
labels = pl(df['question'].to_list())

text_and_labels = pd.concat([df['question'], pd.Series(labels)], axis=1)
df['question'] = text_and_labels.apply(replace_text, axis=1)
print(df)
df.to_csv("sportsQnA_namesreplaced.csv", index=False)

Device set to use cpu


,question,0
0,"From 2019, you guys had multiple games where y...",[]
1,"When you came in here, you kind of had the exp...","[{'entity_group': 'PER', 'score': 0.9974368, '..."
2,"We were discussing about how, when you make a ...",[]
3,"It seems like kind of a curse, is it?",[]
4,"When you did get traded over, (indiscernible) ...","[{'entity_group': 'PER', 'score': 0.9979467, '..."
...,...,...
495,"Austin, walking me through the game winner. W...","[{'entity_group': 'PER', 'score': 0.9858834, '..."
496,I'm curious what that moment was like for you ...,[]
497,"Austin, in the fourth inning you were able to ...","[{'entity_group': 'PER', 'score': 0.9922207, '..."
498,"Ozzie, what did you see on Bellinger's hit and...","[{'entity_group': 'PER', 'score': 0.99599874, ..."


In [ ]:
print(df)
df.to_csv("sportsQnA_namesreplaced.csv", index=False)